# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

프레임워크
-	프레임워크가 정해준 규칙에 따라서 코드를 짜야함
- 원하는 기능 구현에 집중하여 개발할 수 있도록 일정한 형태와 필요한 기능을 갖추고 있는 골격, 뼈대를 의미

라이브러리
-	코드를 위한 도구, 내 마음대로 실행 가능, import 해서 쓰는 것
- 소프트웨어를 개발할 때 컴퓨터 프로그램이 사용하는 비휘발성 자원의 모임. 즉 특정 기능을 모와둔 코드, 함수들의 집합이며 코드 작성 시 활용 가능한 도구

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

딥러닝과 머신러닝 관계
- 딥러닝은 인공신경망 알고리즘을 이용한
머신러닝의 한 분야

딥러닝과 머신러닝 차이
- 머신러닝의 라이브러리는는 전처리 + 모델 돌리기 전 과정을 위한 함수가 많고, 딥러닝 라이브러리는 전처리보다는 모델에 치중, 파라미터가 훨씬 다양

### Q3. 아래의 코드에 주석 달기.

In [ ]:
# 파이토치 관련 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms


# cuda 사용 가능하면 cuda 이용, 안되면 cpu 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

In [ ]:
# 하이퍼파라미터 설정
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [ ]:
#데이터 불러오고 텐서로 바꾸기
#학습 및 테스트 데이터셋
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [ ]:
# 데이터 로드
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)


examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:

class ConvNet(nn.Module):
  # 초기화
  def __init__(self): 
        super(ConvNet, self).__init__() # 상속
        # 레이어들 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 

  # forward
  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # conv1 -> maxpool -> relu 
        x = F.relu(self.mp(self.conv2(x))) # -> conv2 -> maxpool -> relu
        x = self.drop2D(x)  # -> dropout
        x = x.view(x.size(0), -1) # 차원 변경
        x = self.fc1(x) # fully connected
        x = self.fc2(x)
        return F.log_softmax(x) # softmax -> log

In [ ]:

model = ConvNet().to(device) # ConvNet 객체 생성 후 cuda/cpu에 올리기
criterion = nn.CrossEntropyLoss().to(device) # 손실함수 정의
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # optimizer 올리기

In [ ]:
# epoch마다 
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        # data, target 올리기
        data = data.to(device)
        target = target.to(device)
        # gradients를 0으로 초기화
        optimizer.zero_grad()
        # 예측
        hypothesis = model(data)
        # 손실값
        cost = criterion(hypothesis, target)
        # backward
        cost.backward()
        # backward 단계에서 수집된 변화도로 매개변수를 조정
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
# eval
# evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
model.eval()
#evaluation/validation 과정에선 보통 model.eval()과 torch.no_grad()를 함께 사용
with torch.no_grad(): 
  # 맞춤
    correct = 0
  # 전체
    total = 0
  # 정확도 계산
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.